# image_handler

# Recursos

In [317]:
import cv2
import tkinter as tk
import numpy as np
from tkinter import filedialog, Toplevel, Canvas, Button, Frame
from PIL import Image, ImageTk

# Variáveis Globais

In [318]:
# Variáveis globais para gerenciar o estado da imagem
imagem_original = None
imagem_atual = None
historico = []
selection = None  # Coordenadas da área selecionada
caminho = None
cascata = False
canvas = None  # Definir canvas como variável global

# imageController

In [319]:
# Função para abrir a imagem
def abrir_imagem():
    global imagem_original, imagem_atual, caminho, canvas
    
    # Abrir a caixa de diálogo para selecionar uma imagem
    caminho_imagem = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])
    
    if caminho_imagem:
        caminho = caminho_imagem
        imagem_original = Image.open(caminho_imagem)
        imagem_atual = imagem_original
        
        # Converter a imagem para um formato que o Tkinter pode mostrar
        imagem_tk = ImageTk.PhotoImage(imagem_atual)
        
        # Exibir a imagem no canvas
        canvas.create_image(0, 0, anchor="nw", image=imagem_tk)
        canvas.image = imagem_tk  # Manter uma referência para a imagem

# videoController

In [320]:
# Função para abrir o vídeo
def abrir_video():
    global canvas  # Usar a variável global canvas
    
    # Abrir a caixa de diálogo para selecionar um vídeo
    caminho_video = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mov")])
    
    if caminho_video:
        # Usar OpenCV para abrir o vídeo
        cap = cv2.VideoCapture(caminho_video)
        
        def exibir_frame():
            ret, frame = cap.read()
            if ret:
                # Converter o frame de BGR (OpenCV) para RGB (Tkinter)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Converter o frame para uma imagem do Tkinter
                imagem_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                
                # Exibir o frame no canvas
                canvas.create_image(0, 0, anchor="nw", image=imagem_tk)
                canvas.image = imagem_tk  # Manter uma referência para a imagem
                
                # Continuar a mostrar o próximo frame
                canvas.after(30, exibir_frame)  # A cada 30 ms, mostrar o próximo frame
        
        # Iniciar a exibição dos frames
        exibir_frame()

# mainView

In [323]:
# Tela principal
def abrirJanelaPrincipal():
    global canvas, selection  # Tornar canvas acessível na função
    janela_principal = tk.Tk()
    janela_principal.title("Ferramenta de Manipulação de Imagem e Vídeo")
    janela_principal.geometry("1024x768")  # Tamanho inicial
    janela_principal.minsize(800, 600)  # Tamanho mínimo da janela
    janela_principal.resizable(True, True)

    # Configurar o frame principal
    frame = Frame(janela_principal)

    # Configurando o label da página principal
    label = tk.Label(frame, text="Ferramenta de Manipulação de Imagens e Vídeos", font=("Arial", 18), fg="black", bg="lightgray")

    # Canvas para a imagem
    canvas = Canvas(frame, bg="gray")

    #CheckButton para cascata
    cascata_check = tk.BooleanVar()
    check = tk.Checkbutton(frame, text="Ativar Filtro em Cascata", variable=cascata_check)

    # Ativar seleção no canvas
    canvas.bind("<ButtonPress-1>", iniciar_selecao)
    canvas.bind("<B1-Motion>", fazer_selecao)
    canvas.bind("<ButtonRelease-1>", finalizar_selecao)

    # Adicionando nas posições desejadas
    # Row 0
    frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
    label.grid(row=0, column=0, columnspan=10, padx=10, pady=5, sticky="ew")

    # Row 1
    Button(frame, text="Abrir Imagem", command=abrir_imagem).grid(row=1, column=0, padx=10, pady=5, sticky="ew")
    canvas.grid(row=1, column=1, columnspan=10, rowspan=10, sticky="nsew", pady=10)

    # Row 2
    Button(frame, text="Abrir vídeo", command=abrir_video).grid(row=2, column=0, padx=10, pady=5, sticky="ew")

    # Row 3
    check.grid(row=3, column=0, padx=5,pady=10)

    ## Menu de Filtros de Convolução
    # Row 4
    filtrosLabel = tk.Label(frame, text="Filtros de Convolução")
    filtrosLabel.grid(row=4, column=0, padx=5,pady=10)
    # Row 5
    Button(frame, text="Blur").grid(row=5, column=0, padx=10, pady=5, stick="ew")
    # Row 6
    Button(frame, text="Sharpen").grid(row=6, column=0, padx=10, pady=5, stick="ew")
    # Row 7
    Button(frame, text="Emboss").grid(row=7, column=0, padx=10, pady=5, stick="ew")
    # Row 8
    Button(frame, text="Laplacian").grid(row=8, column=0, padx=10, pady=5, stick="ew")
    # Row 9
    Button(frame, text="Canny").grid(row=9, column=0, padx=10, pady=5, stick="ew")
    # Row 10
    Button(frame, text="Sobel").grid(row=10, column=0, padx=10, pady=5, stick="ew")

    ## Menu de Modo de Cor
    # Row 11
    modeCorLabel = tk.Label(frame, text="Modo de Cor")
    modeCorLabel.grid(row=11, column=0, padx=5,pady=10)
    # Row 12
    Button(frame, text="Cinza").grid(row=12, column=0, padx=10, pady=5, stick="ew")
    # Row 13
    Button(frame, text="Binário").grid(row=13, column=0, padx=10, pady=5, stick="ew")
    # Row 14
    Button(frame, text="Cor").grid(row=14, column=0, padx=10, pady=5, stick="ew")

    ## Controles Gerais
    # Row 11
    menuGeralLabel = tk.Label(frame, text="Controles Gerais")
    menuGeralLabel.grid(row=11, column=1, padx=5,pady=10)
    # Row 12
    Button(frame, text="Desfazer", command=desfazer).grid(row=12, column=1, padx=10, pady=5, sticky="ew")
    # Row 13
    Button(frame, text="Original").grid(row=13, column=1, padx=10, pady=5, stick="ew")
    # Row 14
    Button(frame, text="Salvar", command=salvar_imagem).grid(row=14, column=1, padx=10, pady=5, sticky="ew")
    # Row 15
    #Button(frame, text="Fechar").grid(row=15, column=1, padx=10, pady=5, stick="ew")

    ## Opções do Quadro
    # Row 11
    menuQuadroLabel = tk.Label(frame, text="Opções Quadro")
    menuQuadroLabel.grid(row=11, column=2, padx=5,pady=10)
    # Row 12
    Button(frame, text="Zoom na Seleção", command=aplicar_zoom_selecao).grid(row=12, column=2, padx=10, pady=5, sticky="ew")
    # Row 13
    Button(frame, text="Zoom +", command=lambda: aplicar_zoom("mais")).grid(row=13, column=2, padx=10, pady=5, sticky="ew")
    # Row 14
    Button(frame, text="Zoom -", command=lambda: aplicar_zoom("menos")).grid(row=14, column=2, padx=10, pady=5, sticky="ew")

     ## Opções de Vídeo
    # Row 11
    menuVideoLabel = tk.Label(frame, text="Opções Vídeo")
    menuVideoLabel.grid(row=11, column=3, columnspan=2, padx=5,pady=10)
    # Row 12
    Button(frame, text="Reproduzir").grid(row=12, column=3, padx=10, pady=5, stick="ew")
    Button(frame, text="Reverter").grid(row=12, column=4, padx=5, pady=5, stick="ew")
    # Row 13
    Button(frame, text="Acelerar").grid(row=13, column=3, padx=10, pady=5, stick="ew")
    Button(frame, text="Desacelerar").grid(row=13, column=4, padx=5, pady=5, stick="ew")
    # Row 14
    Button(frame, text="Cortar").grid(row=14, column=3, padx=10, pady=5, stick="ew")
    Button(frame, text="Salvar Frame").grid(row=14, column=4, padx=5, pady=5, stick="ew")

    # Inicia a interface
    return janela_principal

# Funções Imagem

In [169]:
# Função para salvar imagem
def salvar_imagem():
    global imagem_atual
    if imagem_atual is None:
        print("Nenhuma imagem carregada para salvar.")
        return

    caminho = abrirTelaSalvarImagem()
    if caminho:
        imagem_atual.save(caminho)
        print(f"Imagem salva em: {caminho}")

# Função para adicionar histórico
def adicionar_historico():
    global imagem_atual, historico
    if imagem_atual:
        historico.append(imagem_atual.copy())

# Função para desfazer
def desfazer():
    global imagem_atual, historico, canvas
    if not historico:
        print("Nenhuma ação para desfazer.")
        return

    imagem_atual = historico.pop()
    exibir_imagem_no_canvas()

# Função para aplicar zoom na área selecionada, mantendo proporções
def aplicar_zoom_selecao():
    global imagem_atual, selection
    if imagem_atual is None:
        print("Nenhuma imagem carregada para aplicar zoom.")
        return

    if not selection:
        print("Nenhuma área selecionada para aplicar zoom.")
        return

    adicionar_historico()
    x1, y1, x2, y2 = selection

    # Certificar-se de que as coordenadas estão dentro dos limites
    largura, altura = imagem_atual.size
    x1, x2 = max(0, min(x1, x2)), min(largura, max(x1, x2))
    y1, y2 = max(0, min(y1, y2)), min(altura, max(y1, y2))

    # Recortar a área selecionada
    area_selecionada = imagem_atual.crop((x1, y1, x2, y2))

    # Calcular nova dimensão preservando proporção
    largura_selecao, altura_selecao = area_selecionada.size
    fator_zoom = 2  # Fator de zoom
    nova_largura = int(largura_selecao * fator_zoom)
    nova_altura = int(altura_selecao * fator_zoom)

    area_ampliada = area_selecionada.resize((nova_largura, nova_altura), Image.Resampling.LANCZOS)

    # Centralizar no Canvas mantendo proporção
    canvas_largura, canvas_altura = 500, 500
    proporcao_area = nova_largura / nova_altura
    proporcao_canvas = canvas_largura / canvas_altura

    if proporcao_area > proporcao_canvas:
        # Ajustar pela largura
        largura_final = canvas_largura
        altura_final = int(largura_final / proporcao_area)
    else:
        # Ajustar pela altura
        altura_final = canvas_altura
        largura_final = int(altura_final * proporcao_area)

    area_final = area_ampliada.resize((largura_final, altura_final), Image.Resampling.LANCZOS)

    # Criar uma nova imagem com fundo cinza e centralizar a área ampliada
    imagem_zoom = Image.new("RGB", (canvas_largura, canvas_altura), "gray")
    offset_x = (canvas_largura - largura_final) // 2
    offset_y = (canvas_altura - altura_final) // 2
    imagem_zoom.paste(area_final, (offset_x, offset_y))

    imagem_atual = imagem_zoom
    exibir_imagem_no_canvas()

# Funções para manipular a seleção no canvas
def iniciar_selecao(event):
    global selection
    selection = [event.x, event.y, event.x, event.y]  # Coordenadas iniciais

def fazer_selecao(event):
    global canvas, selection
    if selection:
        selection[2], selection[3] = event.x, event.y  # Atualizar coordenadas finais
        canvas.delete("selection")  # Remover seleção anterior
        canvas.create_rectangle(
            selection[0], selection[1], selection[2], selection[3], outline="red", tags="selection"
        )

def finalizar_selecao(event):
    global selection
    if selection:
        selection[2], selection[3] = event.x, event.y  # Coordenadas finais

# Exibir imagem no Canvas
def exibir_imagem_no_canvas():
    global imagem_atual, canvas
    if canvas is None or imagem_atual is None:
        return

    imagem_tk = ImageTk.PhotoImage(imagem_atual)
    canvas.img = imagem_tk  # Evita garbage collection
    canvas.delete("all")
    canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

# Função para abrir e exibir a imagem
def abrir_imagem():
    caminho = filedialog.askopenfilename(filetypes=[("Imagens", "*.png;*.jpg;*.jpeg;*.bmp")])
    if caminho:
        inicializar_edicao(caminho)

# Função para inicializar a edição de uma imagem
def inicializar_edicao(caminho):
    global imagem_atual, imagem_original, canvas, historico, selection

    # Limpar histórico e seleção
    historico.clear()
    selection = None

    # Nova janela para exibir a imagem
    janela_imagem = Toplevel()
    janela_imagem.title("Editar Imagem")

    # Configurar o frame principal
    frame = Frame(janela_imagem)
    frame.pack()

    # Canvas para a imagem
    canvas = Canvas(frame, width=500, height=500, bg="gray")
    canvas.grid(row=0, column=0, columnspan=3)

    # Abrir e redimensionar a imagem
    imagem_atual = Image.open(caminho)
    imagem_original = imagem_atual.copy()  # Armazenar a imagem original
    imagem_atual.thumbnail((500, 500))
    historico.append(imagem_atual.copy())
    exibir_imagem_no_canvas()

    # Ativar seleção no canvas
    canvas.bind("<ButtonPress-1>", iniciar_selecao)
    canvas.bind("<B1-Motion>", fazer_selecao)
    canvas.bind("<ButtonRelease-1>", finalizar_selecao)

    # Adicionar controles
    Button(frame, text="Salvar", command=salvar_imagem).grid(row=1, column=0, pady=10)
    Button(frame, text="Desfazer", command=desfazer).grid(row=1, column=1, pady=10)
    Button(frame, text="Zoom na Seleção", command=aplicar_zoom_selecao).grid(row=1, column=2, pady=10)


# video_handler

In [21]:
import cv2
from tkinter import Toplevel, Label, Button, Canvas, filedialog
from PIL import Image, ImageTk

# Variáveis globais para controle de vídeo
video_capture = None
canvas_video = None
frame_atual = None

# Função para abrir e visualizar um vídeo
def abrir_video():
    global video_capture, canvas_video

    # Seleção de arquivo de vídeo
    caminho = filedialog.askopenfilename(filetypes=[("Vídeos", "*.mp4;*.avi;*.mov")])
    if not caminho:
        return

    # Abrir vídeo com OpenCV
    video_capture = cv2.VideoCapture(caminho)
    if not video_capture.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Criar janela de vídeo
    janela_video = Toplevel()
    janela_video.title("Visualizar Vídeo")

    # Canvas para exibir quadros do vídeo
    canvas_video = Canvas(janela_video, width=800, height=600, bg="black")
    canvas_video.pack()

    # Botões de controle
    Button(janela_video, text="Reproduzir", command=reproduzir_video).pack(side="left", padx=5)
    Button(janela_video, text="Capturar Quadro", command=capturar_quadro).pack(side="left", padx=5)
    Button(janela_video, text="Fechar", command=lambda: fechar_video(janela_video)).pack(side="right", padx=5)

def reproduzir_video():
    global video_capture, canvas_video, frame_atual

    if not video_capture or not video_capture.isOpened():
        print("Nenhum vídeo carregado para reprodução.")
        return

    # Ler próximo quadro do vídeo
    ret, frame = video_capture.read()
    if not ret:
        print("Fim do vídeo ou erro.")
        return

    # Converter frame para exibição com PIL
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_atual = ImageTk.PhotoImage(Image.fromarray(frame))

    # Atualizar o Canvas
    canvas_video.create_image(0, 0, anchor="nw", image=frame_atual)
    canvas_video.img = frame_atual  # Previne garbage collection

    # Continuar reprodução
    canvas_video.after(30, reproduzir_video)

def capturar_quadro():
    global frame_atual

    if not frame_atual:
        print("Nenhum quadro disponível para capturar.")
        return

    # Salvar o quadro atual como imagem
    frame_atual._PhotoImage__photo.write("quadro_capturado.png", format="png")
    print("Quadro capturado e salvo como 'quadro_capturado.png'.")

def fechar_video(janela):
    global video_capture

    if video_capture and video_capture.isOpened():
        video_capture.release()
    janela.destroy()

# main_application

In [324]:
def main():
    abrirJanelaPrincipal().mainloop()
main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\josed\anaconda3\envs\teti\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\josed\AppData\Local\Temp\ipykernel_6560\783506861.py", line 93, in <lambda>
    Button(frame, text="Zoom -", command=lambda: aplicar_zoom("menos")).grid(row=14, column=2, padx=10, pady=5, sticky="ew")
                                                 ^^^^^^^^^^^^
NameError: name 'aplicar_zoom' is not defined
